Setup of neo4j from desktop app:

1. create new local db
    Name: SDM_Lab1_Research
    Username: neo4j
    Password: pass1234

2. Start the db


In [35]:
pip install neo4j


Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
import os
from neo4j import GraphDatabase

In [37]:
NEO4J_URI = "bolt://localhost:7687"  # Default for Neo4j Desktop
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pass1234"  # Change this to your actual password

In [45]:
from neo4j import GraphDatabase

def clear_database():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("✅ Database cleared!")

clear_database()


✅ Database cleared!


In [39]:
import subprocess

# Run a shell command
result = subprocess.run(['sh', '-c', 'ls ~/Library/Application\ Support/Neo4j\ Desktop/Application/relate-data/dbmss'], capture_output=True, text=True)

# Get the output
print("Output:", result.stdout)


Output: dbms-6f9440a9-c56d-4ebc-b244-90d92b771350



<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:4: SyntaxWarning: invalid escape sequence '\ '
/var/folders/lr/ysc7n3fx6sv02d60ctrl7k_r0000gn/T/ipykernel_4017/3695089776.py:4: SyntaxWarning: invalid escape sequence '\ '
  result = subprocess.run(['sh', '-c', 'ls ~/Library/Application\ Support/Neo4j\ Desktop/Application/relate-data/dbmss'], capture_output=True, text=True)


In [40]:
# copy the output above in the path here

IMPORT_FOLDER = os.path.expanduser("~/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/")




- Nodes:

1. Paper: doi, title, abstract, year, # citations
2. Author: author id, name
3. Conference: conf id, name 
4. Edition: edition id, edition year, conference id, proceeding name , city
5. Journal: id, name
6. Keyword/Topic: id, name

- Relationship:
1. author - writes -> paper : author id, paper doi
2. author - corresponding author of -> paper : : author id, paper doi
3. paper - cited by -> paper : DOI1, DOI2
4. paper - about -> keyword: doi, keyword id
5. paper - published in ({volume}) -> journel : doi, journal id , volume number , pages, year
6. paper - published in -> conference/workshop edition : doi, edition id , pages
7. edition - of -> conference/workshop : edition id, conf id
8. author - reviews -> paper : : reviewer id "same as author id", paper doi

In [41]:
import shutil
shutil.copy("Paper_nodes.csv", IMPORT_FOLDER)
shutil.copy("Author_nodes.csv", IMPORT_FOLDER)
shutil.copy("fields_of_study.csv", IMPORT_FOLDER)
shutil.copy("conferences.csv", IMPORT_FOLDER)
shutil.copy("journals.csv", IMPORT_FOLDER)
shutil.copy("conference_editions.csv", IMPORT_FOLDER)



'/Users/marwasulaiman/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/conference_editions.csv'

In [43]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

session = driver.session() 

In [46]:
# Load papers 
query =     """
            LOAD CSV WITH HEADERS FROM "file:///Paper_nodes.csv" AS row
            CREATE (:Paper { DOI : row.DOI, title : row.Title, abstract : row.Abstract, year : toInteger(row.Year), 
            publicationDate : date(substring(row.publicationDate, 0, 10)), citationCount : toInteger(row.Citations), referenceCount : toInteger(row.References),
            URL : row.URL})
            """
session.run(query)



In [47]:
# Load Authors 

query =     """
            LOAD CSV WITH HEADERS FROM "file:///Author_nodes.csv" AS row
            CREATE (:Author { ID : toInteger(row.Author_ID), Name: row.Author_Name})
            """
session.run(query)

In [48]:
# Load Keywords 

query =     """
            LOAD CSV WITH HEADERS FROM "file:///fields_of_study.csv" AS row
            CREATE (:Keyword { ID : row.Field_ID, Name: row.Field_Name})
            """
session.run(query)

In [49]:
# Load conferences
query =     """
            LOAD CSV WITH HEADERS FROM "file:///conferences.csv" AS row
            CREATE (:Conference { ID : row.ID, Name: row.Name, URL: row.url})
            """
session.run(query)

In [50]:
# Load Editions
query =     """
            LOAD CSV WITH HEADERS FROM "file:///conference_editions.csv" AS row
            CREATE (:Conference_Edition { ID : row.Edition_ID, Name: row.Conference_Edition_Name, Venue_ID: row.Venue_ID, Year:row.Year})
            """
session.run(query)

In [51]:
# Load Journals
query =     """
            LOAD CSV WITH HEADERS FROM "file:///journals.csv" AS row
            CREATE (:Journal { ID : row.ID, Name: row.Name, ISSN: row.issn, URL: row.url})
            """
session.run(query)

Relationships

In [52]:
shutil.copy("author_paper_relationship_concat.csv", IMPORT_FOLDER)
shutil.copy("paper_field_relationship_concat.csv", IMPORT_FOLDER)
shutil.copy("references.csv", IMPORT_FOLDER)
shutil.copy("reviews.csv", IMPORT_FOLDER)



'/Users/marwasulaiman/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/reviews.csv'

In [53]:
# Writes
query =    """LOAD CSV WITH HEADERS FROM 'file:///author_paper_relationship_concat.csv' AS row
            MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
            MERGE (a)-[w:Writes]->(p)
            """
session.run(query)

In [54]:
# Corresponding
query =    """LOAD CSV WITH HEADERS FROM 'file:///author_paper_relationship_concat.csv' AS row
            MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
            WHERE toBoolean(row.Corresponding) = True
            MERGE (a)-[:Corresponding_of]->(p);  // Creating a separate edge for corresponding authors
            """
session.run(query)

In [20]:
# # Writes (referenced papers)

# query =    """LOAD CSV WITH HEADERS FROM 'file:///ref_author_paper_relationship.csv' AS row
#             MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
#             MERGE (a)-[w:Writes]->(p)
#             """
# session.run(query)

In [21]:
# # Corresponding (referenced papers)

# query =    """LOAD CSV WITH HEADERS FROM 'file:///ref_author_paper_relationship.csv' AS row
#             MATCH (a:Author {ID: toInteger(row.Author_ID)}), (p:Paper {DOI: row.DOI})
#             WHERE toBoolean(row.Corresponding) = True
#             MERGE (a)-[:Corresponding_of]->(p);  // Creating a separate edge for corresponding authors
#             """
# session.run(query)

In [55]:
# Cited by

query =    """LOAD CSV WITH HEADERS FROM 'file:///references.csv' AS row
            MATCH (p:Paper {DOI: row.Paper_DOI}), (r:Paper {DOI: row.Reference_DOI})
            MERGE (p)-[:Cited_by]->(r);  
            """
session.run(query)

In [56]:
# About

query =    """LOAD CSV WITH HEADERS FROM 'file:///paper_field_relationship_concat.csv' AS row
            MATCH (p:Paper {DOI: row.DOI}), (k:Keyword {ID: row.Field_ID})
            MERGE (p)-[:About]->(k);  
            """
session.run(query)

In [24]:
# # About (referenced)

# query =    """LOAD CSV WITH HEADERS FROM 'file:///ref_paper_field_relationship.csv' AS row
#             MATCH (p:Paper {DOI: row.DOI}), (k:Keyword {ID: row.Field_ID})
#             MERGE (p)-[:About]->(k);  
#             """
# session.run(query)

In [57]:
# Edition of

query =    """LOAD CSV WITH HEADERS FROM 'file:///conference_editions.csv' AS row
            MATCH (e:Conference_Edition {ID: row.Edition_ID}), (c:Conference {ID: row.Venue_ID})
            MERGE (e)-[:Of]->(c);  
            """
session.run(query)

In [58]:

# Separate Journal and Conference 

papers_df = pd.read_csv("papers.csv")
referenced_df = pd.read_csv("referenced_papers.csv")

journal_papers_df = papers_df[papers_df["publicationType"]=="journal"]
journal_ref_papers_df = referenced_df[referenced_df["publicationType"]=="journal"]

journal_papers_concat = pd.concat([journal_papers_df,journal_ref_papers_df], ignore_index=True)

conf_papers_df = papers_df[papers_df["publicationType"]=="conference"]
conf_ref_papers_df = referenced_df[referenced_df["publicationType"]=="conference"]

conf_papers_concat = pd.concat([conf_papers_df,conf_ref_papers_df], ignore_index=True)

In [59]:
conf_papers_concat['Edition_id'] = conf_papers_concat['Edition_id'].astype(str)


In [60]:
journal_papers_concat.to_csv("Journal_papers.csv", index=False)
conf_papers_concat.to_csv("Conference_papers.csv", index=False)


In [61]:
shutil.copy("Journal_papers.csv", IMPORT_FOLDER)
shutil.copy("Conference_papers.csv", IMPORT_FOLDER)

'/Users/marwasulaiman/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/Conference_papers.csv'

Note that the referenced papers might have NaN calue for venue id, or edition id , so we might have some papers not matched with editions of journals -> decide whether to filter them out in the first script/ keep them like this/assign to them random venues?

Also, rn we have an edition with "Unknown" venue id, which all papers with "Unknown" venue id matches to

In [62]:
# Published_in

query =    """LOAD CSV WITH HEADERS FROM 'file:///Journal_papers.csv' AS row
            MATCH (p:Paper {DOI: row.DOI}), (j:Journal {ID: row.venue_id})
            MERGE (p)-[r:Published_in]->(j)
            SET r.volume_number = row.volume,
                r.pages = row.pages,
                r.year = toInteger(row.Year);
            """
session.run(query)

In [63]:

query =    """LOAD CSV WITH HEADERS FROM 'file:///Conference_papers.csv' AS row
            MATCH (p:Paper {DOI: row.DOI}), (e:Conference_Edition {ID: row.Edition_id})
            MERGE (p)-[r:Published_in]->(e)
            SET r.pages = row.pages;
            """
session.run(query)

In [64]:
# Reviews

query =    """LOAD CSV WITH HEADERS FROM 'file:///reviews.csv' AS row
            MATCH (a:Author {ID: toInteger(row.Reviewer_ID)}), (p:Paper {DOI: row.Paper_DOI})
            MERGE (a)-[r:Reviews]->(p)
            """
session.run(query)